Acquire

In [70]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

from sklearn.model_selection import train_test_split

from pandas.core.window.ewm import ExponentialMovingWindow as emw

import wrangle
import warnings
warnings.filterwarnings("ignore")

In [2]:
X_df, y_df = wrangle.acquire_amex(sample_size=200000)
X_df = wrangle.clean_amex(X_df)
# wrangle.split_amex(X_df, y_df, train_size=.5, test_size=.5)

Summarize/Verify data

In [3]:
y_df.target.value_counts(normalize=True)

# print('Train: %d rows, %d cols' % y_train.shape)
# print('Validate: %d rows, %d cols' % y_validate.shape)
# print('Test: %d rows, %d cols' % y_validate.shape)

# print('Train: %d rows, %d cols' % X_train.shape)
# print('Validate: %d rows, %d cols' % X_validate.shape)
# print('Test: %d rows, %d cols' % X_test.shape)

# y_df.target.value_counts(normalize=True)

# y_train.target.value_counts(normalize=True)

0    0.741066
1    0.258934
Name: target, dtype: float64

Flatten the time series data. 

For each variable, we need to create the following:



In [71]:
X_df.reset_index(drop=True).set_index(['S_2'])

sample = X_df[X_df.customer_ID == X_df.customer_ID[0]][['customer_ID','P_2']]
sample

,customer_ID,P_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.936665
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.954180
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.960384
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.947248
5,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.945964
6,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.940705
7,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.914767
8,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.950845
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580


In [ ]:
sample.groupby('customer_ID').agg(['min', 'max', 'std', 'mean', 'median', 'count'])

# lambda x: np.polyfit(sample.index, x, 1)[0]
slope = np.polyfit(sample.index, sample.P_2, 1)[0]

r = np.corrcoef(sample.index, sample.P_2, rowvar=False)[0][1]

cv = sample.P_2.std()/sample.P_2.mean()

In [83]:
# those where I will need to take the last available value. The "most recent" value
sample['P_2_ema'] = sample[['P_2']].ewm(alpha=.8, min_periods=1, adjust=True).mean().shift(periods=1)

# rolling standard deviation
sample['P_2_std'] = sample[['P_2']].rolling(window=12, min_periods=1, closed='left').mean()

# compute upper and lower bands, with a weight of 3
k = 3
sample['P_2_ubb'] = sample.P_2_ema + k*sample.P_2_std
sample['P_2_lbb'] = sample.P_2_ema - k*sample.P_2_std

# compute %b and bandwidth
sample['P_2_pctb'] = (sample.P_2 - sample.P_2_lbb) / (sample.P_2_ubb - sample.P_2_lbb)
sample['bandwidth'] = (sample.P_2_ubb - sample.P_2_lbb) / sample.P_2_ema

# rolling difference
n_steps = 2
def diff(sample):
    return sample.iloc[-1] - sample.iloc[0]

sample['P_2_delta'] = sample['P_2'].rolling(window=n_steps).apply(diff)

sample

,customer_ID,P_2,P_2_ema,P_2_rolling_mean,P_2_sd,P_2_ubb,P_2_lbb,P_2_pctb,bandwidth,P_2_delta,P_2_std
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.936665,0.938469,0.938469,0.001276,3.753875,-1.876937,0.499680,6.000000,-0.001804,0.938469
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.954180,0.936965,0.937567,0.011097,3.749665,-1.875735,0.503060,6.003851,0.017516,0.937567
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.960384,0.950848,0.943105,0.011188,3.780162,-1.878465,0.501685,5.951135,0.006203,0.943105
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.947248,0.958489,0.947424,0.008753,3.800762,-1.883784,0.498023,5.930738,-0.013135,0.947424
5,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.945964,0.949494,0.949619,0.006719,3.791661,-1.892674,0.499379,5.986702,-0.001285,0.947389
6,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.940705,0.946669,0.951944,0.006604,3.788124,-1.894785,0.498950,6.003056,-0.005259,0.947152
7,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.914767,0.941898,0.948575,0.018789,3.780589,-1.896794,0.495221,6.027602,-0.025938,0.946231
8,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.950845,0.920193,0.937171,0.018609,3.747086,-1.906700,0.505421,6.144130,0.036078,0.942298
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580,0.944714,0.938070,0.045816,3.774456,-1.885027,0.486547,5.990682,-0.082265,0.943247


,customer_ID,P_2,P_2_ema,P_2_sd,P_2_ubb,P_2_lbb,P_2_pctb,bandwidth
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,0.938469,NaN,NaN,NaN,NaN,NaN
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.936665,0.936965,0.001276,0.940792,0.933138,0.460716,0.008169
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.954180,0.950848,0.011097,0.984141,0.917556,0.550040,0.070027
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.960384,0.958489,0.011188,0.992052,0.924925,0.528228,0.070034
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.947248,0.949494,0.008753,0.975752,0.923235,0.457249,0.055311
5,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.945964,0.946669,0.006719,0.966826,0.926513,0.482492,0.042584
6,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.940705,0.941898,0.006604,0.961709,0.922087,0.469893,0.042066
7,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.914767,0.920193,0.018789,0.976561,0.863825,0.451870,0.122514
8,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.950845,0.944714,0.018609,1.000543,0.888886,0.554903,0.118191
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580,0.883807,0.045816,1.021253,0.746360,0.444608,0.311033


In [26]:
# .agg(['min', 'max', 'std', 'mean', 'median'])
# grouped.transform(lambda x: (x - x.mean()) / x.std())



TypeError: Transform function invalid for data types

Explore the different columns, datatypes, descriptive stats

For reference: 
* D_* = Delinquency variables
* S_* = Spend variables
* P_* = Payment variables
* B_* = Balance variables
* R_* = Risk variables

In [ ]:
spend = X_df.iloc[:,X_df.columns.str[0] == 'S']
delinq = X_df.iloc[:,X_df.columns.str[0] == 'D']
pay = X_df.iloc[:,X_df.columns.str[0] == 'P']
balance = X_df.iloc[:,X_df.columns.str[0] == 'B']
risk = X_df.iloc[:,X_df.columns.str[0] == 'R']

**Spend variables**

- 22 total columns

- S_2: date *needs to be converted* **done**

- All others: float

- S_2, S_5, S_6, S_8, S_11:S_13, S_15:S_20 : no missing values

- S_22:S_26 : missing < 1% of values

- S_3, S_7, S_27 : missing 1-25% of values

- S_9, S_27 : missing 25-75% of values

In [ ]:
spend.info()

**Delinquency Variables**

- 96 total columns

- D_63: Object

- D_64: Object

- All others: float

- D_39, D_47, D_51, D_58, D_60, D_63, D_65, D_71, D_75, D_86, D_92, D_93, D_94, D_96, D_127 : no missing values

- D_42, D_49, D_66, D_73, D_76, D_87, D_88, D_106, D_108, D_110, D_111, D_132, D_134:D_138, D_142 : missing > 75% of values.

- D_41, D_44:D_46, D_48, D_52, D_54:D_55, D_59, D_61, D_62, D_64, D_68:D_70, D_72, D_74, D_78:D_81, D_83, D_84, D_89, D_91, D_102:D_104, D_107, D_109, D_112:D_126, D_128:D_131, D_133, D_139:D_145: missing < 25%

- D_43, D_50, D_53 D_56, D_77, D_82, D_105 : 25-75% missing



In [ ]:
delinq.D_63.value_counts()

In [ ]:
delinq.D_64.value_counts()

In [ ]:
delinq.info()

**Payment Variables**

- 3 total columns (P_2, P_3, P_4)

- all: float

- P_4 : no missing values

- P_2 & P_3 : missing < 1%

In [ ]:
pay.info()

In [ ]:
pay.describe()

**Balance Variables**

- 40 variables

- B_31: int (0, 1)

- all others: float

- B_29, B_39, and B_42 are majority null

- B_17 is missing 

- B_1, B_4, B_5, B_7, B_9, B_10, B_11, B_12, B_14, B_18, B_21, B_23, B_24, B_28, B_31, B_32, B_36 have no missing values. 

- B_2, B_3, B_6, B_8, B_13, B_15, B_16, B_19, B_20, B_25, B_26, B_27, B_30, B_33, B_37, B_38, B_40, B_41 are missing < 1% 


In [ ]:
balance.B_31.value_counts()

In [ ]:
balance.info()

In [ ]:
balance.describe().T

**Risk Variables**

- 28 Columns

- All: float

- R_9, R_26: missing > 90% of values. 

- R_12, R_20, and R_27 are missing < 1%

- R_1:R_8, R_10:R_11, R13:R19, R21:R26, R28 :  no missing values

In [ ]:
risk.info()

In [ ]:
# generate lists of column names by datatype for future use in analysis
object_cols = ['D_63', 'D_64']
int_cols = ['B_31']
date_cols = ['S_2']

# list of non_float columns in order to generate a list of all float column names (186 columns)
non_float_cols = object_cols + int_cols + date_cols
float_cols = [col for col in X_df.columns if col not in non_float_cols]
len(float_cols)

In [ ]:
with pd.option_context('display.max_rows', None,):
    print(null_df.sort_values('total_nulls'))

In [ ]:
null_df.groupby('feature_category').percent_nulls.agg(['mean', 'median', 'max', 'min']).sort_values('mean', ascending=False)